# Strcutured output using GPT-4o models

This cookbook demonstrates how to obtain structured output using GPT-4o models. The OpenAI beta client SDK provides a parse helper that allows you to use your own Pydantic model, eliminating the need to define a JSON schema. This approach is recommended for supported models.

Currently, this feature is supported for:

- gpt-4o-mini on OpenAI
- gpt-4o-2024-08-06 on OpenAI
- gpt-4o-2024-08-06 on Azure

Let's define a simple message type that carries explanation and output for a Math problem

In [1]:
from pydantic import BaseModel


class MathReasoning(BaseModel):
    class Step(BaseModel):
        explanation: str
        output: str

    steps: list[Step]
    final_answer: str

In [2]:
import os

# Set the environment variable
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://YOUR_ENDPOINT_DETAILS.openai.azure.com/"
os.environ["AZURE_OPENAI_API_KEY"] = "YOUR_API_KEY"
os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"] = "gpt-4o-2024-08-06"
os.environ["AZURE_OPENAI_API_VERSION"] = "2024-08-01-preview"

In [4]:
import os
from autogen_core.components.models import AzureOpenAIChatCompletionClient, UserMessage
import json

# Create the client
client = AzureOpenAIChatCompletionClient(
    model=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME"),
    api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    model_capabilities={
        "vision": False,
        "function_calling": True,
        "json_output": True,
    },
)

In [5]:
# Define the user message
messages = [
    UserMessage(content="What is 16 + 32?", source="user"),
]

# Call the create method on the client, passing the messages and additional arguments
# The extra_create_args dictionary includes the response format as MathReasoning model we defined above
# Providing the response format and pydantic model will use the new parse method from beta SDK
response = await client.create(messages=messages, extra_create_args={"response_format": MathReasoning})

# Print the response content after loading it as JSON
print(json.loads(response.content))

# Validate the response content with the MathReasoning model
MathReasoning.model_validate(json.loads(response.content))

{'steps': [{'explanation': 'Start by aligning the numbers vertically.', 'output': '  16\n+32'}, {'explanation': 'Add the digits in the ones column: 6 + 2 = 8.', 'output': '8 in the ones column'}, {'explanation': 'Add the digits in the tens column: 1 + 3 = 4.', 'output': '4 in the tens column'}], 'final_answer': '48'}


MathReasoning(steps=[Step(explanation='Start by aligning the numbers vertically.', output='  16\n+32'), Step(explanation='Add the digits in the ones column: 6 + 2 = 8.', output='8 in the ones column'), Step(explanation='Add the digits in the tens column: 1 + 3 = 4.', output='4 in the tens column')], final_answer='48')